In [ ]:
from pandas.core.frame import DataFrame
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import time
import os

# Importing Files

In [2]:
project_links_df = pd.read_excel('project_links.xlsx')
project_links_df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'project_links.xlsx'

In [3]:
# creating a new column for the code in the project link

project_links_df['Project Link Code'] = project_links_df['Project Link'].apply(lambda x: x.split("/")[-1])
project_links_df.head()

,Project Link,Project Approval Date,Project Code,County,Project Name,Project Status,Open Action Items,Project Link Code
0,https://data.archwaypartnership.uga.edu/projec...,NaN,H-217,Hart,Feasibility Study for Recreation Center at the...,Completed,NaN,3858
1,https://data.archwaypartnership.uga.edu/node/3705,NaN,H-184,Hart,Hart Botanical Garden Brochure,Completed,NaN,3705
2,https://data.archwaypartnership.uga.edu/projec...,NaN,H-232,Hart,Historic Downtown Hartwell Ambassador Project ...,Completed,NaN,3944
3,https://data.archwaypartnership.uga.edu/projec...,NaN,CA-73,Candler,Historical Marker Project,Completed,NaN,3661
4,https://data.archwaypartnership.uga.edu/projec...,NaN,HB-34,Habersham,"""Hidden Gems of Habersham"" Brochure Project",Cancelled,NaN,3231


### Function to rename files

In [4]:
project_links_df['Project Link Code'].iloc[0:10]

0    3858
1    3705
2    3944
3    3661
4    3231
5    3154
6    3674
7    2651
8    2652
9    2653
Name: Project Link Code, dtype: object

In [5]:
search_url = "https://data.archwaypartnership.uga.edu/"

browser = webdriver.Chrome()

# Open browser to begin search
browser.get(search_url)

# Maximize the window
browser.maximize_window()

time.sleep(2)

In [6]:
# entering the username
input_username = browser.find_element(By.ID,'edit-name')
input_username.send_keys("bs83243")

time.sleep(1)

# entering the password
input_password = browser.find_element(By.ID,'edit-pass')
input_password.send_keys("BS@15july1997")


time.sleep(1)

# clicking the login button
button_login = browser.find_element(By.ID,'edit-submit')
button_login.click()
time.sleep(2)

In [7]:

# Load the CSV file with your data
df = project_links_df

# Iterate over the rows in the dataframe
for index, row in df.iterrows():
    code = str(row['Project Link Code'])
    project_link = row['Project Link']
    print(f'Processing project link: {project_link}')
    browser.get(project_link)
    time.sleep(2)

    # deliverable_folder = f'../../Downloads/Test-Rename/{code}-deliverables'
    deliverable_folder = f'/Users/bsingh/Downloads/Test-rename/{code}-deliverables'
    
    next_page_exists = True
    while(next_page_exists):
       next_page_exists = False
       # Get the HTML content to find the <a> tag containing the file name
       html_content = browser.page_source
       table = browser.find_elements(By.TAG_NAME, 'table')
       if(len(table) < 4):
           print(f"No deliverables for: {project_link}")
           break
       tbody = table[3].find_elements(By.TAG_NAME, 'tbody')[0]
       rows = tbody.find_elements(By.TAG_NAME, 'tr')

       for row in rows:
           data = row.find_elements(By.TAG_NAME, 'td')[0]
           a_tag = data.find_element(By.TAG_NAME, 'a')
           del_link = a_tag.get_attribute('href')

           old_file_name = del_link.split('/')[-1]
           new_file_name = data.text
        #    Check is old file name is present
           if os.path.isfile(f'{deliverable_folder}/{old_file_name}'):
               os.rename(f'{deliverable_folder}/{old_file_name}', f'{deliverable_folder}/{new_file_name}')
               print(f'Renamed {old_file_name} to {new_file_name}')
           
       #    Check for next page , complex code
       #    Find the navigation panel at the bottom, not present for projects with only one page
       nav = browser.find_elements(By.CLASS_NAME, 'pager')
       if len(nav) > 0:
        #    Find for the li in the ul and check every li for the next page button, if exist then click it
            nav_list = nav[0].find_elements(By.TAG_NAME, 'li')
            for li in nav_list:
                a_tag = li.find_elements(By.TAG_NAME, 'a')
                if a_tag and 'Go to next page' in a_tag[0].get_attribute('title'):
                    a_tag[0].click()
                    time.sleep(2)
                    next_page_exists = True
                    break
                
    break

Processing project link: https://data.archwaypartnership.uga.edu/project/3858
